In [18]:
from qrnn import get_model, Qloss, qloss
from keras.callbacks import *
import numpy as np

# Generate the synthetic data
x1 = np.sin(np.arange(0, 9, 0.01))
x2 = np.cos(np.arange(0, 9, 0.01))
x3 = x1**2
x4 = (x1+x2)/2

Xtrain = np.vstack((x2, x3, x4)).T #(900, 3)
Ytrain = np.array([x1]*1).T #(900, 1)

In [20]:
# Parameters
input_dim = 3
num_hidden_layers = 2
num_units = [200, 200]
act = ['relu', 'relu']
dropout = [0.1, 0.1]
gauss_std = [0.3, 0.3]

# Get model
model = get_model(input_dim, num_units, act, dropout, gauss_std, num_hidden_layers)
print(model.summary())

# Train
early_stopping = EarlyStopping(monitor='val_qloss_score', patience=5)
model.compile(loss=qloss, optimizer='adam', metrics=[Qloss()])
model.fit(x=Xtrain, y=Ytrain, 
          epochs=10, 
          validation_split=0.2, 
          batch_size=64, 
          shuffle=True, 
          callbacks=[early_stopping]
         )

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 200)               800       
_________________________________________________________________
dropout_14 (Dropout)         (None, 200)               0         
_________________________________________________________________
gaussian_noise_14 (GaussianN (None, 200)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_15 (Dropout)         (None, 200)               0         
_________________________________________________________________
gaussian_noise_15 (GaussianN (None, 200)             